# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
part1_df = pd.read_csv("clean_city_data.csv")
part1_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Roald,100,NO,1594698246,87,62.58,6.13,53.60,3.36
1,1,Maltahöhe,0,NaN,1594698246,87,-24.83,16.98,34.36,3.67
2,2,Tuktoyaktuk,20,CA,1594698247,52,69.45,-133.04,64.40,5.82
3,3,Atuona,44,PF,1594698120,78,-9.80,-139.03,79.36,16.46
4,4,Saint-Joseph,37,RE,1594698100,68,-21.37,55.62,66.20,4.70
...,...,...,...,...,...,...,...,...,...,...
541,554,Gogrial,45,SS,1594698371,89,8.53,28.10,71.83,3.29
542,555,Rjukan,100,NO,1594698371,99,59.88,8.59,54.00,5.01
543,556,Jacareacanga,82,BR,1594698371,90,-6.22,-57.75,73.49,0.07
544,557,Menongue,0,AO,1594698372,57,-14.66,17.69,43.39,2.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = part1_df[["Latitude","Longitude"]]
humidity = part1_df["Humidity"]
gmap_figure = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=humidity, point_radius=3) 
gmap_figure.add_layer(heatmap)
gmap_figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_weather = part1_df.loc[(part1_df["Max Temp"]<80)&(part1_df["Max Temp"]>70)&(part1_df["Wind Speed"]<10)&(part1_df["Cloudiness"]==0)].dropna()
new_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
54,54,Ekibastuz,0,KZ,1594698258,50,51.67,75.37,76.37,9.04
199,203,Tūkrah,0,LY,1594698173,82,32.53,20.58,75.69,9.33
222,227,Santa Isabel Rio Negro,0,BR,1594698129,88,-0.41,-65.02,73.08,1.70
291,297,Roquetas de Mar,0,ES,1594698312,53,36.76,-2.61,77.00,8.05
323,331,Maţāy,0,EG,1594698319,66,28.42,30.78,73.06,7.25
327,335,Mehrān,0,IR,1594698320,25,33.12,46.16,77.00,4.70
334,342,Tiznit Province,0,MA,1594698043,58,29.58,-9.50,71.83,2.13
390,400,La Palma del Condado,0,ES,1594698336,73,37.39,-6.55,75.20,4.70
406,416,Sergeevka,0,KZ,1594698339,62,53.88,67.42,76.17,1.23
449,459,Contamana,0,PE,1594698349,89,-7.33,-75.02,72.59,1.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = new_weather[["City","Country","Latitude","Longitude"]].copy()
hotel_df["Hotel Name"] =""
hotel_param = {"radius":5000,"keyword":"hotel","key":g_key}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    hotel_param["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
  # make request and print url
    name_address = requests.get(base_url, params=hotel_param).json()
    pprint.pprint(name_address)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 51.7125918, 'lng': 75.3373451},
                           'viewport': {'northeast': {'lat': 51.71395707989272,
                                                      'lng': 75.33878187989272},
                                        'southwest': {'lat': 51.71125742010728,
                                                      'lng': 75.33608222010729}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '34aa20f91cdd7e7a7d8173c05a7409c98f3d0df0',
              'name': 'Amiliya',
              'photos': [{'height': 1824,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/110762316780479104470">A '
                                                'Senkebayev</a>'],
                          'photo_reference': 'CmRaAAAA

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 36.725, 'lng': -2.625},
                           'viewport': {'northeast': {'lat': 36.72636137989272,
                                                      'lng': -2.623818370107278},
                                        'southwest': {'lat': 36.72366172010727,
                                                      'lng': -2.626518029892722}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '1acb6f2d404fcca31ef2b34ad947e5af1eeb2ba3',
              'name': 'Playacapricho Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1417,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/106459333271444502417">Playacapricho '
                                                'Hot

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 37.36191400000001,
                                        'lng': -6.541161},
                           'viewport': {'northeast': {'lat': 37.36330932989272,
                                                      'lng': -6.539724070107277},
                                        'southwest': {'lat': 37.36060967010727,
                                                      'lng': -6.542423729892721}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '29eda537dd6dc137e23455b018ec9442423075bd',
              'nam

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.


In [8]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
54,Ekibastuz,KZ,51.67,75.37,Amiliya
199,Tūkrah,LY,32.53,20.58,مطعم الحيطة
222,Santa Isabel Rio Negro,BR,-0.41,-65.02,Hotel Ayuana
291,Roquetas de Mar,ES,36.76,-2.61,Playacapricho Hotel
323,Maţāy,EG,28.42,30.78,
327,Mehrān,IR,33.12,46.16,
334,Tiznit Province,MA,29.58,-9.50,
390,La Palma del Condado,ES,37.39,-6.55,Hotel León
406,Sergeevka,KZ,53.88,67.42,
449,Contamana,PE,-7.33,-75.02,Hotel Zavala´S


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_markers)

# Display figure
fig


NameError: name 'fig' is not defined